In [1]:
import sys
#!{sys.executable} -m pip install efficientnet
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
# Get pretrained model 
from tensorflow.keras.applications.efficientnet import EfficientNetB0,preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image

# Get Connection
import boto3
from boto3 import Session

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Handle Images
from PIL import Image
from io import BytesIO
import io

session = Session()
credentials = session.get_credentials()

#aws access credentials
current_credentials = credentials.get_frozen_credentials()

client = boto3.client(
    's3',
    aws_access_key_id=current_credentials.access_key,
    aws_secret_access_key=current_credentials.secret_key,
    aws_session_token=current_credentials.token)

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [3]:
bucket_name = 'tellisa-collage-gan'

In [4]:
def get_s3_keys(bucket):
    """Get a list of keys in an S3 bucket."""
    keys = []
    resp = client.list_objects_v2(Bucket=bucket)
    for obj in resp['Contents']:
        keys.append(obj['Key'])
    return keys

In [5]:
collageKeys = np.array(get_s3_keys(bucket_name))
collageKeys = collageKeys[14:]

In [6]:
len(collageKeys) #limited to pulling 1000 objects

986

In [7]:
import concurrent.futures 



def get_image(img_key):
    bucket_name = 'tellisa-collage-gan'
    img_inst = client.get_object(Bucket=bucket_name,Key=img_key)
    #img_inst = Image.open(io.BytesIO(img_inst['Body'].read()))
    #print('step 1')
    #model = EfficientNetB0(include_top = False, weights = 'imagenet', pooling='avg')
    #x = image.img_to_array(img_inst)
    #x = np.expand_dims(x,axis=0)
    #x = preprocess_input(x)
    #print(x)
    #preds = model.predict(x)
    #curr_df = pd.DataFrame(preds[0]).T
    return img_inst['Body'].read()



In [8]:
imageMetadata = []
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(get_image,collageKeys)
    for result in results:
        imageMetadata.append(result)

In [9]:
model = EfficientNetB0(include_top = False, weights = 'imagenet', pooling='avg')

def get_image_vector(img):
    img = Image.open(io.BytesIO(img))  #(img['Body'].read()))
    x = image.img_to_array(img)
    x = np.expand_dims(x,axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    return pd.DataFrame(preds[0]).T

In [10]:
imageVectorFrame = pd.DataFrame()
numIters = 0 
for i in imageMetadata:
    imageVectorFrame = imageVectorFrame.append(get_image_vector(i)).reset_index(drop=True)
    if numIters%100==0:
        print(numIters)
    numIters+=1

0
100
200
300
400
500
600
700
800
900


In [11]:
imageVectorFrame

,0,1,2,3,4,5,6,7,8,9,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,-0.073504,-0.081529,-0.150256,-0.039397,-0.196335,-0.058430,0.089128,-0.125144,-0.161661,-0.073682,...,-0.047790,-0.013145,0.001945,-0.036383,-0.208646,-0.152522,-0.067945,-0.050996,0.012269,-0.137268
1,-0.139749,0.065019,-0.103396,0.045213,0.049502,0.782018,0.078437,-0.067619,0.265398,0.007390,...,0.076177,0.134568,-0.130283,-0.071177,-0.042576,0.047607,0.232933,0.075863,0.197335,0.723809
2,0.045302,-0.000459,-0.135456,-0.122611,-0.127929,0.521024,-0.045943,-0.141464,0.125021,-0.034702,...,0.192965,-0.036452,-0.015031,-0.088858,0.292089,0.337222,0.057235,-0.113923,-0.064440,0.202605
3,-0.029537,0.258865,-0.110284,0.350379,0.003658,0.724655,0.097174,-0.080560,1.121670,0.201151,...,-0.080352,-0.034084,0.338343,0.185316,0.295396,0.007392,0.274381,0.205001,-0.007172,0.544511
4,0.391050,-0.089162,-0.087924,0.546654,-0.048115,-0.033974,0.539756,-0.120903,0.191909,-0.070164,...,0.073363,0.156908,-0.022547,0.106340,-0.017653,0.334020,-0.024949,-0.004910,0.153678,0.963530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,-0.035328,-0.051516,-0.175977,-0.056747,-0.106838,0.386835,-0.071185,-0.087870,-0.043810,-0.051170,...,-0.116426,-0.115446,-0.094607,-0.067893,-0.161541,-0.226661,-0.054693,-0.154794,-0.084642,0.008974
982,0.810265,-0.049186,0.430618,-0.111807,-0.178421,0.097249,0.044609,-0.136758,0.050721,-0.082266,...,-0.078689,-0.086006,-0.038312,-0.131303,0.261286,0.230873,-0.061863,-0.136276,0.062419,0.220637
983,-0.067020,-0.061268,-0.137906,-0.100233,-0.018890,0.033099,0.152568,-0.155820,0.273074,-0.062739,...,-0.074611,-0.122182,-0.042912,-0.021819,0.337572,0.297257,-0.110470,0.021228,-0.088385,0.060651
984,-0.152389,-0.039377,0.391156,-0.109111,-0.166413,-0.012149,-0.019238,-0.129766,-0.100565,0.023961,...,-0.053926,-0.150955,-0.130605,-0.077299,0.168750,0.078739,-0.077848,-0.146313,-0.037919,-0.215715
